In [20]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 
from data_loader import dataloader
user,ratings,movies = dataloader()
ratings = ratings.drop("timestamp",axis=1)
movie_index_by_id = {id: i for i, id in enumerate(movies["movie_id"])}


In [21]:
class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
            learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        # regularization parameter
        self.lam = lam
        # learning rate for gradient descent
        self.learning_rate = learning_rate
        # maximum number of iterations
        self.max_iter = max_iter
        # print results after print_every iterations
        self.print_every = print_every
        # user-based or item-based
        self.user_based = user_based
        # number of users, items, and ratings. 
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = 3901
        self.n_ratings = Y_data.shape[0]
        
        if Xinit is None: # new
            self.X = np.random.randn(self.n_items, K)
        else: # or from saved data
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: # from daved data
            self.W = Winit
            
        # normalized data, update later in normalized_Y function
        self.Y_data_n = self.Y_raw_data.copy()


    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users

        # if we want to normalize based on item, just switch first two columns of data
        else: # item bas
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
    def loss(self):
        L = 0 
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
        
        # take average
        L /= self.n_ratings
        # regularization, don't ever forget this 
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 
    def get_items_rated_by_user(self, user_id):
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
            
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item item_id and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            # gradient
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            # gradient
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        # truncate if results are out of range [0, 5]
        if pred < 0:
            return 0 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        """
        predict ratings one user give all unrated items
        """
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((y_pred[i],i))
        
        return predicted_ratings
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE

In [22]:
ratings = ratings.values
print(ratings)

[[   1 1193    5]
 [   1  661    3]
 [   1  914    3]
 ...
 [6040  562    5]
 [6040 1096    4]
 [6040 1097    4]]


In [23]:
ratings1 = ratings.copy()

In [24]:
ratings[:,1]= [movie_index_by_id[movie_id] for movie_id in ratings1[:,1]]


In [25]:
print(ratings)
a= np.max(ratings[:,1])

print(a)

[[   1 1176    5]
 [   1  655    3]
 [   1  902    3]
 ...
 [6040  558    5]
 [6040 1080    4]
 [6040 1081    4]]
3882


In [26]:
user_prof1 = pd.read_csv('u_dict.csv')
print(user_prof1.dtypes)
user_prof1 = user_prof1.values
print(user_prof1.shape)

user_id    int64
genres     int64
weight     int64
dtype: object
(93887, 3)


In [27]:
print(ratings[:, 0])
ratings[:, 0] -= 1
user_prof1[:, 0] -= 1
# ratings = np.concatenate((ratings,user_prof1), axis = 0)
# print(ratings[:, :2])
from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print(rate_train.shape)

print(len(rate_train))

rs = MF(rate_train, K = 2, lam = 0.1, print_every = 1, learning_rate = 1, max_iter = 12, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

[   1    1    1 ... 6040 6040 6040]
(670140, 3)
670140
iter = 1 , loss = 9.901396735495833 , RMSE train = 1.4121427463080085
iter = 2 , loss = 8.771027616646554 , RMSE train = 1.3164245633573763
iter = 3 , loss = 7.810803091585528 , RMSE train = 1.2383998140014283
iter = 4 , loss = 6.983934169947945 , RMSE train = 1.1776342746146022
iter = 5 , loss = 6.264179878787339 , RMSE train = 1.131974323309666
iter = 6 , loss = 5.632396394994861 , RMSE train = 1.0987826144404627
iter = 7 , loss = 5.074271614004013 , RMSE train = 1.0752824651691133
iter = 8 , loss = 4.5788297737049435 , RMSE train = 1.059026247434094
iter = 9 , loss = 4.137438121607559 , RMSE train = 1.047980929661515
iter = 10 , loss = 3.7431420594596108 , RMSE train = 1.0405786375207517
iter = 11 , loss = 3.390216003692198 , RMSE train = 1.0356669895762214
iter = 12 , loss = 3.0738564936968373 , RMSE train = 1.032423509594745

Item-based MF, RMSE = 1.0438527700494604


In [28]:

z = movie_index_by_id.copy()
z=list(z.items())
print(z[0][1])

0


In [29]:
a = rs.pred_for_user(1999)
a.sort(reverse= True)
for i in range(10):
    print(a[i][0],"-",movies.iloc[z[ a[i][1]][1],1],"-",movies.iloc[z[a[i][1]][1],2])

3.9770209730049957 - Ideal Husband, An (1999) - Comedy
3.9550648947663856 - Nueba Yol (1995) - Comedy|Drama
3.951812720888385 - 39 Steps, The (1935) - Thriller
3.950058226030006 - See the Sea (Regarde la mer) (1997) - Drama
3.9465630719892255 - Empire Records (1995) - Comedy|Drama
3.9377888064499733 - Species (1995) - Horror|Sci-Fi
3.9376958882597375 - Mondo (1996) - Drama
3.937154625174276 - Taxman (1999) - Comedy|Drama
3.936620699959415 - Three Wishes (1995) - Drama
3.9347278890078656 - French Kiss (1995) - Comedy|Romance


In [30]:
import pickle
filename = 'rs_mf.sav'
pickle.dump(rs,open(filename,'wb'))

In [31]:
import pickle
rs1 = pickle.load((open('rs_mf.sav','rb')))
z = movie_index_by_id.copy()
z=list(z.items())
print(z[0][1])

0


In [32]:
b = rs1.pred_for_user(16)
b.sort(reverse= True)
for i in range(10):
    print(b[i][0],"-",movies.iloc[z[ b[i][1]][1],1],"-",movies.iloc[z[b[i][1]][1],2])


4.252810904994349 - Freddy's Dead: The Final Nightmare (1991) - Horror
4.242999194500042 - Monsieur Verdoux (1947) - Comedy
4.2384560431819605 - Best Laid Plans (1999) - Crime|Drama
4.225854194278971 - Big Daddy (1999) - Comedy
4.2251006482363636 - Thomas Crown Affair, The (1968) - Crime|Drama|Thriller
4.2169785263075905 - Predator (1987) - Action|Sci-Fi|Thriller
4.216742142780604 - Problem Child 2 (1991) - Comedy
4.21483099893079 - Blackbeard's Ghost (1968) - Children's|Comedy
4.214209008424528 - Romancing the Stone (1984) - Action|Adventure|Comedy|Romance
4.207704349054404 - Night Visitor, The (1970) - Crime|Thriller


In [33]:
b = rs1.pred_for_user(18)
b.sort(reverse= True)
for i in range(10):
    print(b[i][0],"-",movies.iloc[z[ b[i][1]][1],1],"-",movies.iloc[z[b[i][1]][1],2])

4.0590536548939085 - Boys on the Side (1995) - Comedy|Drama
4.025246343809984 - Music From Another Room (1998) - Drama|Romance
3.972988153337556 - Last Man Standing (1996) - Action|Drama|Western
3.968429495749117 - Hell in the Pacific (1968) - Drama|War
3.957887656781839 - Scrooged (1988) - Comedy
3.9510290081506194 - Boys, Les (1997) - Comedy
3.9506371141443726 - Jacob's Ladder (1990) - Horror|Mystery|Thriller
3.9465621330405716 - Replacement Killers, The (1998) - Action|Thriller
3.9312910390654063 - Delicatessen (1991) - Comedy|Sci-Fi
3.929995407252556 - Dumbo (1941) - Animation|Children's|Musical


In [34]:
import time
start = time.time()
for i in range(1000):
    rs1.pred_for_user(i)
elapsed = (time.time() - start)/1000
print(elapsed)
    

0.00822639513015747
